<a href="https://colab.research.google.com/github/Debantam-Majilla/Bridgeweave-Assignment/blob/main/Customer_Online_Time_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings                        # To ignore any warnings
warnings.filterwarnings("ignore")
%matplotlib inline
%pylab inline

import os
import pandas as pd
import numpy as np

import glob 
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format = 'retina'

Populating the interactive namespace from numpy and matplotlib


**Importing Customer Dataset**


In [ ]:
customer=pd.read_csv("/customers.csv")
print("Shape of customers table :",customer.shape)
print("\n")
print(customer.head())

Shape of customers table : (2500, 4)


       id gender  age  number_of_kids
0  979863   MALE   26               2
1  780123   MALE   60               2
2  614848   MALE   45               4
3  775046   MALE   62               3
4  991601   MALE   23               0


**Checking for any null values.**

-Turns out- No null values .Customer Data is well structured.

In [ ]:
customer.isnull().sum()

id                0
gender            0
age               0
number_of_kids    0
dtype: int64

Importing ping dataset 


1.   **Timestamps need to convert to Datetime format.**
2.   **We can segment hour portion from datetime format**
3.   **We can understand On which hour of days, most log-ins happened.**
4.   **We can use No of Logins at past for each id ,to be a feature of our prediction.**



In [ ]:
ping=pd.read_csv("/content/sample_data/pings.csv")
print("Shape of customers table :",ping.shape)
print("\n")

#ping1 is for later use
ping1=ping

Shape of customers table : (9029404, 2)




In [ ]:
import dateutil.parser
## convert the timestamp to a python date object 
ping['Time'] = list(map(lambda v : datetime.datetime.fromtimestamp(v).strftime('%Y-%m-%d %H:%M:%S'), ping['timestamp']))
ping['times'] = list(map(dateutil.parser.parse, ping['Time']))

ping['hour'] = list(map(lambda v : v.hour, ping['times']))
ping['day'] = list(map(lambda v : v.day, ping['times']))
ping.head()

**Below cell is giving us- Mostly users loggedin at 2PM or 3 PM of any day.**

In [ ]:
aggregatedData = ping[["hour","day","id"]].groupby(['hour', 'day']).agg(['count'])
    
aggregatedData = aggregatedData.reset_index()
aggregatedData.columns = ['hour','day','app_openings']
aggregatedData

#From this we can understand that users generrally logs in to our app mostly at 2PM or 3 PM

,hour,day,app_openings
0,1,1,54871
1,2,1,174913
2,3,1,177994
3,4,23,1


**Taking No of Log-in per ID as a feature for our model**

In [ ]:
no_logins=ping1.groupby(["id"])["timestamp"].count().reset_index()
no_logins.columns=["id","No_Of_Past_logins"]
no_logins

,id,No_Of_Past_logins
0,111556,1202
1,111575,1636
2,111779,1588
3,111839,7824
4,112486,1613
...,...,...
2470,997190,2306
2471,997500,4090
2472,997510,5161
2473,998215,2273


**Importing test dataset (supervised dataset)**

In [ ]:
test=pd.read_csv("/test.csv")
test

,id,date,online_hours
0,979863,28/06/17,7
1,979863,27/06/17,9
2,979863,26/06/17,9
3,979863,25/06/17,10
4,979863,24/06/17,9
...,...,...,...
17495,165666,26/06/17,7
17496,165666,25/06/17,7
17497,165666,24/06/17,10
17498,165666,23/06/17,6


**Merging Customer table , Test Table & newly generated No of Log_in table for each ID**

In [ ]:
df1=pd.merge(customer,test)

import datetime as dt
df1['Date'] = pd.to_datetime(df1['date'])

final=pd.merge(no_logins,df1).iloc[:,[0,2,3,4,7,1,6]]
final

,id,gender,age,number_of_kids,Date,No_Of_Past_logins,online_hours
0,111556,FEMALE,49,4,2017-06-28,1202,3
1,111556,FEMALE,49,4,2017-06-27,1202,3
2,111556,FEMALE,49,4,2017-06-26,1202,0
3,111556,FEMALE,49,4,2017-06-25,1202,0
4,111556,FEMALE,49,4,2017-06-24,1202,0
...,...,...,...,...,...,...,...
17383,998229,MALE,31,0,2017-06-26,5423,5
17384,998229,MALE,31,0,2017-06-25,5423,5
17385,998229,MALE,31,0,2017-06-24,5423,9
17386,998229,MALE,31,0,2017-06-23,5423,6


**Fetching X-feature vector & Y- output (online_hours)**

In [ ]:
X=final.iloc[:,:-1]
Y=final.iloc[:,6]


**Taking Date as any feature is not a good idea.**
**converting date to numeric values**

In [ ]:
X['Date']=X['Date'].map(dt.datetime.toordinal)

#convert gender column to one hot encoding
gender=pd.get_dummies(X["gender"],drop_first=True)

# drop gender column 
X.drop("gender",axis=1,inplace=True)

#concat the dummy variables
X=pd.concat([X,gender],axis=1)


X


,id,age,number_of_kids,Date,No_Of_Past_logins,MALE
0,111556,49,4,736508,1202,0
1,111556,49,4,736507,1202,0
2,111556,49,4,736506,1202,0
3,111556,49,4,736505,1202,0
4,111556,49,4,736504,1202,0
...,...,...,...,...,...,...
17383,998229,31,0,736506,5423,1
17384,998229,31,0,736505,5423,1
17385,998229,31,0,736504,5423,1
17386,998229,31,0,736503,5423,1


In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

# Fitting Multiple Linear Regression to the Training set
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Predicting the Test set results
y_pred = regressor.predict(X_test)

from sklearn.metrics import r2_score
score=r2_score(y_test,y_pred)

In [ ]:
score

0.3931253343637646

In [ ]:
from sklearn.metrics import mean_squared_error

rms = mean_squared_error(y_test,y_pred, squared=False)
rms

2.637536591333218

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

from sklearn.tree import DecisionTreeRegressor
#DecisionTreeRegressor class has many parameters. Input only #random_state=0 or 42.
regressor = DecisionTreeRegressor(random_state=0)
#Fit the regressor object to the dataset.
regressor.fit(X,Y)

y_pred1 = regressor.predict(X_test)

from sklearn.metrics import r2_score
score1=r2_score(y_test,y_pred1)
print("R-SQURE ERROR IS :",score1)

rms1 = mean_squared_error(y_test,y_pred1, squared=False)
print("RMS ERROR IS ",rms1)

R-SQURE ERROR IS : 0.9986957051272398
RMS ERROR IS  0.12227477799652739
